# 품질검사전문기관 상세정보

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질검사전문기관 상세정보"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/품질검사전문기관 상세정보/selectIoApiPmQtsc.csv'

In [2]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [4]:
JSONKEY = "items"
DUMMY = 0

In [5]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1

In [7]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/품질검사전문기관 목록/selectIoApiPmQtscList.csv",low_memory=False, encoding="ms949")

In [12]:
dfCombi = imsiDf.loc[:,["rgsseq"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen

540

In [9]:
resultDfMerged = pd.DataFrame()

In [13]:
numOfRows = 1000
resultDf = pd.DataFrame()



for i in range(0,dfCombiLen):
    BASEPARAM={"serviceKey":APIKEY, "rgsSeq": dfCombi.loc[i].rgsseq, "type":"json"}
    #print("BASEPARAM ",BASEPARAM)
    resultDf = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
    if resultDf.empty: # 정상 데이터가 없는 경우
        print("No Data")
        break
    else:
        resultDfMerged = resultDfMerged.append(resultDf)


1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2015056, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2015090, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2013006, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2019010, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF',

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 1996002, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2010020, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2014005, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2014037, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2015055, 't

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2010014, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2005005, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2015014, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2007014, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2016001, 't

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2008041, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2013007, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2017010, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2018020, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2015057, 't

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2001002, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2006006, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2007015, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2000001, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2010002, 't

...

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2008018, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2014107, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2013011, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2014117, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2006010, 't

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2016008, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2008022, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2010015, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2012017, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2010001, 't

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2015068, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2010016, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2015005, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2013010, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2010013, 't

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2014082, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2008044, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2014110, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2014102, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2014098, 't

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2021013, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2020002, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2021006, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2021010, 'type': 'json'} rows: 15 completed
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'rgsSeq': 2021009, 't

In [14]:
resultDfMerged = resultDfMerged.drop_duplicates()
resultDfMerged

,rgsseq,tpincts,perno,bsnrngcts,rgsdt,mtlnm,tpinno,rprnnm,tlno,zc,adr,fxno,lbrtare,orcd,orcdnm
0,2015056,None,서울-4-3호,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",20160622,(재)한국건설품질연구원,None,김인식,02-501-5561,462714,경기도 성남시 중원구 갈마치로244번길 31 (상대원동),None,161.90,B1000,None
0,2015090,품질검사(일반,인천-3-2호,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",20151110,(재)한국화학융합시험연구원 인천,None,권오정,032-5783-6800,404817,인천광역시 서구 가재울로 68 (가좌동),None,443.00,B4000,None
0,2013006,특수,제2013-01호,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",20130603,(주)건설재료시험연구소,None,남궁 혁,043-716-1122,368903,충청북도 증평군 광장로 482,043-716-1123,127.50,A3000,None
0,2019010,품질검사(일반,충북-3-12호,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",20150825,(주)건설재료시험연구소,None,성시완,043-716-1122,368903,충청북도 음성군 맹동면 용촌길 41,None,225.00,C9000,None
0,2015059,품질검사(토목,충북-3-3호,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",20150825,(주)건설재료시험연구소,None,성시완,043-716-1122,368903,충청북도 음성군 맹동면 용촌길 41,None,225.00,C9000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2022006,품질검사(특수),부산-3-17호,"용접(방사선비파괴검사),용접(자기비파괴검사),용접(초음파비파괴검사),용접(침투비파괴검사)",20220302,케이티이 주식회사,None,김윤길,051-207-0715,49422,"부산광역시 사하구 비봉로 23, 1층(신평동, 품질회관)",None,51.48,B2000,None
0,2022007,품질검사(특수),인천-3-5호,"용접(방사선비파괴검사),용접(자기비파괴검사),용접(초음파비파괴검사),용접(침투비파괴검사)",20150324,에이원기술검사(주),None,박덕영,032-715-7087,22723,"인천광역시 서구 승학로299번길 3 (연희동, 에이원빌딩)",None,45.90,B4000,None
0,2022008,품질검사(특수),서울-3-32호,"용접(방사선비파괴검사),용접(자기비파괴검사),용접(초음파비파괴검사),용접(침투비파괴검사)",20220317,주식회사 올네이션,None,김종호,070-4346-3110,06249,"서울특별시 강남구 역삼로 152, 5층(역삼동, 올네이션빌딩)",None,65.30,B1000,None
0,2022010,품질검사(일반,부산-3-14호,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재",20141002,(주)한국품질연구원,None,이미지,051-941-6461,46702,부산광역시 강서구 공항로1309번가길 51 (대저1동),None,508.00,B2000,None


In [15]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

품질검사전문기관 상세정보 save compled


In [16]:
resultDfMerged.shape

(540, 15)